In [6]:
import pandas as pd 
from pulp import *

In [24]:
# Read the first few rows dataset in a Pandas DataFrame
# Read only the nutrition info not the bounds/constraints
df = pd.read_excel("diet - medium.xls",nrows=17)

# Create a list of the food items
food_items = list(df['Foods'])

# Create a dictinary of costs for all food items
costs = dict(zip(food_items,df['Price/Serving']))

# Create a dictionary of calories for all food items
calories = dict(zip(food_items,df['Calories']))

# Create a dictionary of total fat for all food items
fat = dict(zip(food_items,df['Total_Fat (g)']))

# Create a dictionary of carbohydrates for all food items
carbs = dict(zip(food_items,df['Carbohydrates (g)']))

fiber = dict(zip(food_items,df['Dietary_Fiber (g)']))

protein = dict(zip(food_items,df['Protein (g)']))

In [55]:
carbs

{'Frozen Broccoli': 13.6,
 'Frozen Corn': 17.1,
 'Raw Lettuce Iceberg': 0.4,
 ' Baked Potatoes': 39.9,
 'Tofu': 2.2,
 'Roasted Chicken': 0.0,
 'Spaghetti W/ Sauce': 58.3,
 'Raw Apple': 21.0,
 'Banana': 26.7,
 'Wheat Bread': 12.4,
 'White Bread': 11.8,
 'Oatmeal Cookies': 12.4,
 'Apple Pie': 9.6,
 'Scrambled Eggs': 1.3,
 'Turkey Bologna': 0.3,
 'Beef Frankfurter': 0.8,
 'Chocolate Chip Cookies': 9.3}

In [25]:
df.head()

,Foods,Price/Serving,Serving Size,Calories,Cholesterol (mg),Total_Fat (g),Sodium (mg),Carbohydrates (g),Dietary_Fiber (g),Protein (g),Vit_A (IU),Vit_C (IU),Calcium (mg),Iron (mg)
0,Frozen Broccoli,0.48,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,Frozen Corn,0.54,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
2,Raw Lettuce Iceberg,0.06,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
3,Baked Potatoes,0.18,1/2 Cup,171.5,0.0,0.2,15.2,39.9,3.2,3.7,0.0,15.6,22.7,4.3
4,Tofu,0.93,1/4 block,88.2,0.0,5.5,8.1,2.2,1.4,9.4,98.6,0.1,121.8,6.2


In [47]:
prob = LpProblem("Simple_Diet_Problem",LpMinimize)

In [48]:
food_vars = LpVariable.dicts("Food",food_items,lowBound=0,cat='Continuous')

In [49]:
prob += lpSum([costs[i]*food_vars[i] for i in food_items])

In [50]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0

In [51]:
# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0, "CarbsMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 60.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 125.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 100.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 150.0, "ProteinMaximum"


In [52]:
prob.solve()

1

In [53]:
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

Food_Frozen_Broccoli = 6.9242113
Food_Scrambled_Eggs = 6.060891
Food__Baked_Potatoes = 1.0806324


In [54]:
value(prob.objective)

5.518229286